# Wissensaggregator Mittelalter und frühe Neuzeit

WIAGweb2  
Daten für Utrechter Priester aus Excel einlesen

[Neuen Typ anlegen](#Neuen-Typ-anlegen)  
[Literatur](#Literatur)    
Priester  
- [Einträge in `item`](#Einträge-in-item)
- [Daten eintragen](#Daten-eintragen)
- [Freie Attribute](#Freie-Attribute)
- [Literaturverweise eintragen](#Literaturverweise-eintragen)
- [Namensvarianten eintragen](#Namensvarianten-eintragen)
- [Suchvarianten eintragen (`name_lookup`)](#Suchvarianten-eintragen-(name_lookup))

[Ämter](#Ämter)

## Vorspann

Modul laden

In [1]:
wds_path = "../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
# using Revise

In [6]:
using WiagDataSetup

In [7]:
Wds = WiagDataSetup

WiagDataSetup

In [8]:
using DataFrames, CSV, Dates

In [9]:
ENV["COLUMNS"] = 120 # Anzeige/Ausgabe für die Inhalte von DataFrames vergrößern

120

In [10]:
computer = :bkroege_goe
computer = :ghertko_kn
if computer == :bkroege_goe
    data_path = "../data"
    out_path = "../data_sql"
else
    data_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data"
    out_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql"
end

"C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql"

In [11]:
item_type_id = 10

10

## Daten löschen
z.B. für ein komplettes Update

In [ ]:
sql_item_ids = "SELECT ID FROM item WHERE item_type_id = $(item_type_id)"
sql = "
DELETE FROM person_role WHERE person_id in ($(sql_item_ids));
DELETE FROM name_lookup WHERE person_id in ($(sql_item_ids));
DELETE FROM person WHERE id in ($(sql_item_ids));
DELETE FROM item_property WHERE item_id in ($(sql_item_ids));
DELETE FROM item_reference WHERE item_id in ($(sql_item_ids));
DELETE FROM item WHERE item_type_id = $(item_type_id);
"
file_name = joinpath(out_path, "delete_item_type_$(item_type_id).sql")
open(file_name, "w") do io
    println(io, sql)
end

## Neuen Typ anlegen

(einmalig)

einlesen aus CSV

In [ ]:
file_name = joinpath(data_path, "item_type.csv");
df_item_type = CSV.read(file_name, DataFrame, delim = ';')

In [ ]:
create_date(date_time) = typeof(date_time) == DateTime ? date_time : DateTime(date_time, dateformat"dd.mm.yy HH:MM")

In [ ]:
transform!(df_item_type, :date_created => ByRow(create_date) => :date_created);

In [ ]:
transform!(df_item_type, :date_changed => ByRow(create_date) => :date_changed);

In [ ]:
names(df_item_type)

alternativ: direkt erstellen

In [ ]:
df_item_type = DataFrame();

In [ ]:
insertcols!(df_item_type,
    :id => item_type_id,
    :comment => missing,
    :name => "Priester Utrecht",
    :note => "Priester des Bistums Utrecht nach Stapel",
    :table_name => "person",
    :id_public_key => "Pers-PRIESTUtrecht",
    :name_app => "priest_ut",
    :created_by => 2,
    :date_created => now(),
    :changed_by => 2,
    :date_changed => now(),
)

In [ ]:
file_name = joinpath(out_path, "item_type.sql")
Wds.insert_sql(file_name, "item_type", df_item_type)

## Literatur

In [12]:
df_reference_volume = DataFrame();

In [13]:
insertcols!(df_reference_volume,
    :id => missing, # wird automatisch vergeben
    :comment => missing,
    :full_citation => "Stapel, R.J. (2020) Ordinations of priests in the medieval diocese of Utrecht (1505-1518). IISH Data Collection.",
    :title_short => "Stapel, Ordinations of priests in the medieval diocese of Utrecht (1505-1518)",
    :author_editor => "Rombert Stapel",
    :note => missing,
    :ri_opac_id => missing,
    :isbn => missing,
    :year_publication => "2020",
    :display_order => 1,
    :gs_volume_nr => missing,
    :gs_doi => missing,
    :gs_url => missing,
    :online_resource => "https://hdl.handle.net/10622/IEYIUX",
    :reference_id => 1,
    :item_type_id => item_type_id
)

,id,comment,full_citation
,Missing,Missing,String
1,missing,missing,"Stapel, R.J. (2020) Ordinations of priests in the medieval diocese of Utrecht (1505-1518). IISH Data Collection."


In [14]:
file_name = joinpath(out_path, "reference_volume.sql")
Wds.insert_sql(file_name, "reference_volume", df_reference_volume)

1

## Einträge in `item`

Daten für die Priester einlesen.

Exportiere die Daten aus Excel über "Speichern unter" im Format "CSV UTF-8" (Auswahlfeld Dateityp)

In [ ]:
file_name = joinpath(data_path, "Ordinations_of_priests_Stapel_BK.csv");
df_priests = CSV.read(file_name, DataFrame, delim = ';', stringtype = String);

In [ ]:
Wds.clean_up!(df_priests)

In [ ]:
size(df_priests, 1)

In [ ]:
df_priests = unique(df_priests);

Der Datensatz einhält Zeilen, die in allen Feldern übereinstimmen.

In [ ]:
size(df_priests, 1)

In [ ]:
names(df_priests)

In [ ]:
columns = [
    :Priester_ID, 
    :Datum, 
    :Titel, 
    Symbol("Vorname (Voornaam)"), 
    Symbol("(Familienname) Familienaam")
]

In [ ]:
df_priests[100:105, columns]

In [ ]:
columns = [
    :Priester_ID => :id_in_source,
]

In [ ]:
df_item = select(df_priests, columns);

Verwende nur den Nummern-Teil der ID für die WIAG-ID

In [ ]:
get_id_num(id_long) = replace(id_long, "id_" => "")

In [ ]:
transform!(df_item, :id_in_source => ByRow(get_id_num) => :id_num_in_source);

In [ ]:
maximum(length.(df_item.id_num_in_source))

In [ ]:
id_public_key = df_item_type[1, :id_public_key]

In [ ]:
function make_id_public(id)        
    num_id_length = 10
    return "WIAG-" * id_public_key * "-" * lpad(id, num_id_length, '0')
end

In [ ]:
transform!(df_item, :id_num_in_source => ByRow(make_id_public) => :id_public);

In [ ]:
df_item[200:205, :]

Der Wert für `lang` ist das Kürzel für die Sprache der Quelle also `nl`?!

In [ ]:
insertcols!(df_item,
    :id => missing,
    :item_type_id => item_type_id,
    :comment => missing,
    :merged_into_id => missing,
    :edit_status => "online",
    :created_by => 2,
    :date_created => now(),
    :changed_by => 2,
    :date_changed => now(),
    :is_deleted => 0,
    :normdata_edited_by => missing,
    :lang => "nl",
    :is_online => 1
);

In [ ]:
df_item[200:205, :]

In [ ]:
table_name = "item"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, select(df_item, Not(:id_num_in_source)))

Importiere `item.sql` via phpMyAdmin in die Datenbank.  

## Daten eintragen

Exportiere die Tabelle `item` aus der Datenbank via phpMyAdmin mit den Optionen: "Spaltennamen in die erste Zeile setzen" und "Spalten trennen mit ';'" als Trennzeichen.

In [12]:
if computer == :bkroege_goe 
    file_name = joinpath(data_path, "item.csv");
else
    file_name = joinpath(data_path, "item_local.csv")
end
df_item_db = CSV.read(file_name, DataFrame, delim = ';');

In [13]:
file_name = joinpath(data_path, "Ordinations_of_priests_Stapel_BK.csv");
df_priests = CSV.read(file_name, DataFrame, delim = ';', stringtype = String);

In [14]:
Wds.clean_up!(df_priests)

In [15]:
size(df_priests, 1)

5255

In [16]:
df_priests = unique(df_priests);

In [17]:
df_item_utrecht = subset(df_item_db, :item_type_id => ByRow(isequal(item_type_id)));

In [18]:
size(df_item_utrecht, 1)

3958

In [19]:
df_idx = df_item_utrecht[:, [:id, :id_in_source, :item_type_id]];

In [20]:
names(df_priests)

11-element Vector{String}:
 "Priester_ID"
 "Datum"
 "Ort (Locatie)"
 "Titel"
 "Vorname (Voornaam)"
 "Zweiter Vorname (Patroniem)"
 "(Familienname) Familienaam"
 "Herkunftsort (Toponiem)"
 "Orden (Orde)"
 "Amt (Functie)"
 "Instelling"

In [21]:
columns = [
    :Priester_ID => :id_in_source,
    :Datum => :ordination_date,
    Symbol("Ort (Locatie)") => :ordination_place,
    :Titel => :title,
    Symbol("Vorname (Voornaam)") => :givenname,
    Symbol("Zweiter Vorname (Patroniem)") => :patronym,
    Symbol("(Familienname) Familienaam") => :familyname,
    Symbol("Herkunftsort (Toponiem)") => :place_of_origin,        
    Symbol("Orden (Orde)") => :religious_order,
    Symbol("Amt (Functie)") => :role_name,
    :Instelling => :institution
];

In [22]:
df_person_in = innerjoin(
    select(df_priests, columns),
    df_idx,
    on = :id_in_source
);

In [23]:
size(df_person_in, 1), size(df_priests, 1), size(df_idx, 1)

(3958, 3958, 3958)

### Identifiziere Orden

In [46]:
file_name = joinpath(data_path, "religious_order.csv")
df_ro = CSV.read(file_name, DataFrame, delim = ";");

In [47]:
df_item_ro = subset(df_item_db, :item_type_id => ByRow(isequal(7)));

In [48]:
df_ro_id = innerjoin(df_ro, select(df_item_db, [:id, :item_type_id, :id_in_source]), on = :id);

Info

In [49]:
df_ro_id

,id,comment,name,abbreviation,note,item_type_id,id_in_source
,Int64,String31?,Missing,String7?,Missing,Int64,String15
1,5309,missing,missing,OP,missing,7,13
2,5310,missing,missing,OFM,missing,7,7
3,5311,missing,missing,OPraem,missing,7,14
4,5312,missing,missing,OT,missing,7,22
5,5313,missing,missing,OSB,missing,7,16
6,5314,missing,missing,OCist,missing,7,5
7,5315,missing,missing,OESA,missing,7,106
8,5316,missing,missing,OCarm,missing,7,3
9,5317,missing,missing,CanA,missing,7,1


In [50]:
file_name = joinpath(data_path, "mapping_orden_stapel.csv")
df_map_ro = CSV.read(file_name, DataFrame, delim = ";");

In [51]:
transform!(df_map_ro, :id_order => ByRow(string) => :id_in_source);

In [52]:
df_map_ro = innerjoin(df_map_ro, df_ro_id, on = :id_in_source);

In [53]:
names(df_map_ro)

10-element Vector{String}:
 "altLabel_abk"
 "Orde"
 "id_order"
 "id_in_source"
 "id"
 "comment"
 "name"
 "abbreviation"
 "note"
 "item_type_id"

In [54]:
columns = [
    :id => :religious_order_id,
    :id_in_source => :id_in_source_ro,
    :Orde => :religious_order,
    :abbreviation => :abbreviation,
]

4-element Vector{Pair{Symbol, Symbol}}:
           :id => :religious_order_id
 :id_in_source => :id_in_source_ro
         :Orde => :religious_order
 :abbreviation => :abbreviation

In [55]:
df_map_ro_join = select(df_map_ro, columns);

Die Ordensbezeichnungen variieren in Groß- und Kleinschreibung

In [56]:
transform!(df_map_ro_join, :religious_order => ByRow(lowercase) => :religious_order);

In [57]:
df_map_ro_join[7:12, :]

,religious_order_id,id_in_source_ro,religious_order,abbreviation
,Int64,String,String,String7?
1,5314,5,ordinis cistericiensis,OCist
2,5314,5,ordinis cistersiensium,OCist
3,176167,6,ordinis sancti wilhelmi,OESG
4,176167,6,ordinis wilhellemitarum,OESG
5,5319,12,ordinis sancti johannis,OMel
6,5309,13,ordinis predicatorum,OP


In [58]:
lowercase_maybe(a) = typeof(a) <: AbstractString ? lowercase(a) : a

lowercase_maybe (generic function with 1 method)

In [59]:
transform!(df_person_in, :religious_order => ByRow(lowercase_maybe) => :religious_order);

In [60]:
df_person_in_ro = leftjoin(df_person_in, df_map_ro_join, on = :religious_order, matchmissing = :notequal);

In [61]:
size(df_person_in, 1), size(df_person_in_ro, 1)

(3958, 3958)

In [62]:
count(!ismissing, df_person_in.religious_order), count(!ismissing, df_person_in_ro.religious_order_id)

(1022, 1007)

In [63]:
delta_msg(a, b) = !ismissing(a) && ismissing(b)

delta_msg (generic function with 1 method)

In [64]:
df_examine = subset(df_person_in_ro, [:religious_order, :religious_order_id] => ByRow(delta_msg));

In [65]:
unique(df_examine.religious_order)

3-element Vector{Union{Missing, String}}:
 "ordinis …"
 "kanoniker"
 "kanoniker ecclesie"

Exportiere Daten für die Tabelle `person`

In [66]:
names(df_person_in_ro)

16-element Vector{String}:
 "id_in_source"
 "ordination_date"
 "ordination_place"
 "title"
 "givenname"
 "patronym"
 "familyname"
 "place_of_origin"
 "religious_order"
 "role_name"
 "institution"
 "id"
 "item_type_id"
 "religious_order_id"
 "id_in_source_ro"
 "abbreviation"

Verwende das Jahr der Priesterweihe für die zeitliche Einordnung

In [67]:
parse_date_min(s) = ismissing(s) ? s : year(s)

parse_date_min (generic function with 1 method)

In [68]:
transform!(df_person_in_ro, :ordination_date => ByRow(parse_date_min) => :date_min);

In [69]:
columns = [
    :id => :id,
    :item_type_id => :item_type_id,
    :givenname => :givenname,
    :familyname => :familyname,
    :religious_order_id => :religious_order_id,
    :date_min => :date_min,
    :date_min => :date_max,
];

In [72]:
is_test(id) = id in (180293, 180301)

is_test (generic function with 1 method)

In [73]:
subset(df_person_in_ro[:, [:id, :givenname, :date_min, :ordination_date, :title]], :id => ByRow(is_test))

,id,givenname,date_min,ordination_date,title
,Int64,String,Int64,Date,String?
1,180301,Johannes,1508,1508-09-16,Frater
2,180293,Amelius,1505,1505-02-08,Magister


In [ ]:
table_name = "person"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, select(df_person_in_ro, columns))

## Freie Attribute

### Titel

In [25]:
df_title = subset(df_person_in, :title => ByRow(!ismissing));

In [26]:
size(df_title)

(1712, 13)

In [27]:
df_item_prop_title = DataFrame();

In [28]:
insertcols!(df_item_prop_title,
    :item_id => df_title.id,
    :name => "title",
    :value => df_title.title
);

In [29]:
df_item_prop_title[70:75, :]

,item_id,name,value
,Int64,String,String?
1,180293,title,Magister
2,180295,title,Frater
3,180296,title,Frater
4,180297,title,Frater
5,180299,title,Frater
6,180301,title,Frater


### Weihe Priester

Weiheamt

In [30]:
df_ordination = subset(df_person_in, :id => ByRow(!ismissing));
size(df_ordination)

(3958, 13)

In [31]:
df_item_prop_ord = DataFrame();

In [32]:
insertcols!(df_item_prop_ord,
    :item_id => df_ordination.id,
    :name => "ordination",
    :value => "Priester"
);

Datum der Weihe

In [74]:
df_ord_date = subset(df_person_in, :ordination_date => ByRow(!ismissing));
size(df_ord_date)

(3958, 13)

In [78]:
df_item_prop_ord_date = DataFrame();

In [79]:
insertcols!(df_item_prop_ord_date,
    :item_id => df_ord_date.id,
    :name => "ordination_date",
    :value => string.(df_ord_date.ordination_date),
);

In [80]:
subset(df_item_prop_ord_date, :item_id => ByRow(is_test))

,item_id,name,value
,Int64,String,String
1,180293,ordination_date,1505-02-08
2,180301,ordination_date,1508-09-16


Ort der Weihe

In [36]:
df_ord_place = subset(df_person_in, :ordination_place => ByRow(!ismissing));
size(df_ord_place)

(3958, 13)

In [37]:
df_item_prop_ord_place = DataFrame();

In [38]:
insertcols!(df_item_prop_ord_place,
    :item_id => df_ord_place.id,
    :name => "ordination_place",
    :value => df_ord_place.ordination_place,
);

Tabelle `item_property` schreiben

In [81]:
df_item_prop = vcat(df_item_prop_title, df_item_prop_ord, df_item_prop_ord_date, df_item_prop_ord_place);

In [40]:
size(df_item_prop)

(13586, 3)

In [82]:
table_name = "item_property"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_item_prop)

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1579
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1579
┌ Info: row 
│   i = 6000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1579
┌ Info: row 
│   i = 8000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1579
┌ Info: row 
│   i = 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1579
┌ Info: row 
│   i = 12000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1579


13586

### Geburtsort
2022-04-12: Trage zunächst nur den Namen ein. Die Zuordnung zu identifizierten Orten ist unklar.

In [24]:
names(df_person_in)

13-element Vector{String}:
 "id_in_source"
 "ordination_date"
 "ordination_place"
 "title"
 "givenname"
 "patronym"
 "familyname"
 "place_of_origin"
 "religious_order"
 "role_name"
 "institution"
 "id"
 "item_type_id"

In [43]:
df_birthplace = subset(df_person_in, :place_of_origin => ByRow(!ismissing));
size(df_birthplace)

(3897, 13)

In [45]:
df_item_prop_birthplace = DataFrame();

In [46]:
insertcols!(df_item_prop_birthplace,
    :item_id => df_birthplace.id,
    :name => "birthplace",
    :value => df_birthplace.place_of_origin,
);

In [49]:
table_name = "item_property"
file_name = joinpath(out_path, "$(table_name)_birthplace.sql")
Wds.insert_sql(file_name, table_name, df_item_prop_birthplace)

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1579


3897

## Literaturverweise eintragen
*R bk* Befülle `id_in_reference` mit 'id_1509_3_83' ... ?

In [ ]:
df_item_reference = DataFrame();

In [ ]:
insertcols!(df_item_reference,
    :reference_id => 1,
    :id_in_reference => df_person_in.id_in_source,
    :item_id => df_person_in.id,
    :item_type_id => item_type_id
);

In [ ]:
df_item_reference[1500:1505, :]

In [ ]:
table_name = "item_reference"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_item_reference)

## Namensvarianten eintragen
Schreibe Patronym (Vatersnamen) in Tabelle `item_property`

In [83]:
df_patronym = subset(df_person_in, :patronym => ByRow(!ismissing));

In [84]:
df_patronym[70:75, [:id, :givenname, :patronym]]

,id,givenname,patronym
,Int64,String,String?
1,180242,Sulfridus,Hilconis
2,180243,Johannes,Johannis
3,180244,Aelbertus,Johannis
4,180245,Judocus,Theodrici
5,180246,Rodolphus,Johannis
6,180248,Rodolplms,Gerardi


In [85]:
df_item_prop_pym = DataFrame()

""
""


In [86]:
insertcols!(df_item_prop_pym,
    :item_id => df_patronym.id,
    :name => "patronym",
    :value => df_patronym.patronym
);

In [87]:
df_item_prop_pym[70:75, :]

,item_id,name,value
,Int64,String,String?
1,180242,patronym,Hilconis
2,180243,patronym,Johannis
3,180244,patronym,Johannis
4,180245,patronym,Theodrici
5,180246,patronym,Johannis
6,180248,patronym,Gerardi


In [88]:
table_name = "item_property"
file_name = joinpath(out_path, "$(table_name)_patronym.sql")
Wds.insert_sql(file_name, table_name, df_item_prop_pym)

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1579


2593

### Suchvarianten eintragen in `name_lookup`.
Das Patronym entspricht für die Suche am ehesten einem zweiten Vornamen.

In [ ]:
df_person_name = select(df_person_in, [:id, :givenname, :patronym, :familyname]);

In [ ]:
function join_space(a...)
    c_a = filter(!ismissing, collect(a))
    return join(c_a, " ")
end

In [ ]:
df_name_variant = transform(df_person_name, [:givenname, :patronym] => ByRow(join_space) => :givenname);

In [ ]:
df_name_variant[50:55, :]

Füge technisch erforderliche Spalten ein. Erzeuge Einträge für `name_lookup`.

In [ ]:
insertcols!(df_name_variant,
    :givenname_variant => missing,
    :familyname_variant => missing,
    :prefix_name => missing,
    :lang => "nl"
);

In [ ]:
df_nl = Wds.create_name_lookup(df_name_variant);

Info

In [ ]:
df_gerardus = subset(df_name_variant, :id => ByRow(isequal(203857)))

In [ ]:
df_nl[55:60, :]

In [ ]:
names(df_nl)

In [ ]:
table_name = "name_lookup"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_nl)

## Ämter

In [ ]:
file_name = joinpath(data_path, "item.csv");
df_item_db = CSV.read(file_name, DataFrame, delim = ';');

In [ ]:
file_name = joinpath(data_path, "Ordinations_of_priests_Stapel_BK.csv");
df_priests = CSV.read(file_name, DataFrame, delim = ';', stringtype = String);

In [ ]:
Wds.clean_up!(df_priests)

In [ ]:
size(df_priests, 1)

In [ ]:
df_priests = unique(df_priests);

In [ ]:
df_item_utrecht = subset(df_item_db, :item_type_id => ByRow(isequal(10)));

In [ ]:
size(df_item_utrecht, 1)

In [ ]:
columns = [
    :id => :person_id,
    :id_in_source => :id_in_source,
    :item_type_id => :item_type_id,
]

In [ ]:
df_idx = select(df_item_utrecht, columns);

In [ ]:
columns = [
    :Priester_ID => :id_in_source,
    :Datum => :ordination_date,
    Symbol("Ort (Locatie)") => :ordination_place,
    :Titel => :title,
    Symbol("Vorname (Voornaam)") => :givenname,
    Symbol("Zweiter Vorname (Patroniem)") => :patronym,
    Symbol("(Familienname) Familienaam") => :familyname,
    Symbol("Herkunftsort (Toponiem)") => :place_of_origin,        
    Symbol("Orden (Orde)") => :religious_order,
    Symbol("Amt (Functie)") => :role_name,
    :Instelling => :institution
];

In [ ]:
names(df_idx)

In [ ]:
df_priests_with_role = subset(select(df_priests, columns), :role_name => ByRow(!ismissing));

In [ ]:
df_person_role = innerjoin(
    df_priests_with_role,
    df_idx,
    on = :id_in_source
);

In [ ]:
size(df_person_role, 1), size(df_priests, 1), size(df_idx, 1)

Setze Diözese 'Utrecht', ID = 53

In [ ]:
insertcols!(df_person_role, :diocese_id => 53, :diocese_name => "Utrecht");

In [ ]:
df_person_role[30:35, [:id_in_source, :givenname, :familyname, :role_name, :diocese_name, :diocese_id]]

### Rollen zuordnen

In [ ]:
file_name = joinpath(data_path, "role.csv");
df_role = CSV.read(file_name, DataFrame);

In [ ]:
names(df_role)

In [ ]:
df_person_role = leftjoin(df_person_role, 
    select(df_role, [:id => :role_id, :name => :role_name_norm]), 
    on = :role_name => :role_name_norm);

In [ ]:
count(ismissing, df_person_role.role_id)

In [ ]:
columns = [
    :person_id, :id_in_source, :title, :givenname, :patronym, :familyname, :role_name, :role_id
];

In [ ]:
subset(select(df_person_role, columns), :role_id => ByRow(ismissing))

In [ ]:
names(df_person_role)

In [ ]:
columns = [
    :person_id => :person_id,
    :role_id => :role_id,
    :role_name => :role_name,
    :diocese_id => :diocese_id,
    :diocese_name => :diocese_name,
    :institution => :institution_name,
];

In [ ]:
df_person_role_out = select(df_person_role, columns);

In [ ]:
table_name = "person_role"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_person_role_out)

Dateien zusammenfügen

In [ ]:
function cat_sql(file_out, file_in...; path = ".")    
    open(joinpath(path, file_out), "w") do io_out
        for file_loop in file_in            
            open(joinpath(path, file_loop)) do io_in
                write(io_out, read(io_in))
            end
        end
    end
end

Inhalte, die unabhängig sind von einzelnen Personen: `item_type.sql`, `reference_volume.sql`

Inhalte für die einzelnen Priester. (Tabelle `item` muss schon im Prozess geschrieben und gelesen worden sein).

In [ ]:
file_out = "item_x.sql"
cat_sql(file_out,
    "item_property.sql",
    "item_property_patronym.sql",
    "item_reference.sql",
    path = out_path,
)

In [ ]:
file_out = "person_x.sql"
cat_sql(file_out,
    "person.sql",
    "person_role.sql",
    "name_lookup.sql",
    path = out_path,
)